In [102]:
import pandas as pd
import json
from datetime import datetime
import holidays
import os
from datetime import timedelta

In [103]:
#df filtered by houston, needed to filter by state = TX (NOT IMPLEMENTED)

subway = pd.read_csv('Subway_Houston_days.csv', encoding="utf-8")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7,8,9,17,19,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [106]:
def is_weekend(df):
    df['date'] = pd.to_datetime(df['date']) 

    df['is_weekend'] = df['date'].dt.day_name()
    df.loc[df['is_weekend'] == "Saturday", 'is_weekend'] = 1
    df.loc[df['is_weekend'] == "Sunday", 'is_weekend'] = 1
    df.loc[df['is_weekend'] == "Monday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Tuesday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Wednesday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Thursday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Friday", 'is_weekend'] = 0

    return df

def is_holiday(df):
    holi = holidays.CountryHoliday('US', prov="Houston", state='TX')
    df["is_holiday"] = [1 if d in holi else 0 for d in df["date"]]
    return df

def last_day(df):
    stores = set(df['placekey'])
    i = 0
    for store in stores:
        new = df[df['placekey'] == store]
        l = []
        
        #for row in new.iterrows():
        yesterday = new['date'] - timedelta(days = 1)
        yesterday = yesterday.dt.strftime('%Y-%m-%d')
        set_raro = set(new['date'].dt.strftime('%Y-%m-%d'))
        for day in yesterday:
            if day in set_raro:
                visits = list(new[new['date'] == day]['visits'])
                l.append(visits[0])
                
            else:
                l.append(0)
        
        new['last_day'] = l

        if i == 0:
            old = new
        else:
            old = pd.concat([old, new])
    return old

def last_week(df):
    stores = set(df['placekey'])
    i = 0
    for store in stores:
        new = df[df['placekey'] == store]
        l = []
        
        #for row in new.iterrows():
        yesterday = new['date'] - timedelta(days = 7)
        yesterday = yesterday.dt.strftime('%Y-%m-%d')
        set_raro = set(new['date'].dt.strftime('%Y-%m-%d'))
        for day in yesterday:
            if day in set_raro:
                visits = list(new[new['date'] == day]['visits'])
                l.append(visits[0])
                
            else:
                l.append(0)
        
        new['last_week'] = l

        if i == 0:
            old = new
        else:
            old = pd.concat([old, new])
    return old

subway = is_weekend(subway)
subway = is_holiday(subway)
subway = last_day(subway)
subway = last_week(subway)


In [107]:
subway

,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,category_tags,...,visitor_daytime_cbgs,date,month,year,visits,day,is_weekend,is_holiday,last_day,last_week
837,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,"{""482014328024"":8,""481576714001"":6,""4820143290...",2020-01-01,1,2020,2,1,0,1,0,0
838,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,"{""482014328024"":8,""481576714001"":6,""4820143290...",2020-01-02,1,2020,11,2,0,0,2,0
839,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,"{""482014328024"":8,""481576714001"":6,""4820143290...",2020-01-03,1,2020,9,3,0,0,11,0
840,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,"{""482014328024"":8,""481576714001"":6,""4820143290...",2020-01-04,1,2020,10,4,1,0,9,0
841,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,"{""482014328024"":8,""481576714001"":6,""4820143290...",2020-01-05,1,2020,2,5,1,0,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88451,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,NaN,NaN,NaN,...,"{""482015313003"":5,""482014211021"":4,""4816772230...",2021-03-27,3,2021,4,27,1,0,4,2
88452,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,NaN,NaN,NaN,...,"{""482015313003"":5,""482014211021"":4,""4816772230...",2021-03-28,3,2021,2,28,1,0,4,0
88453,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,NaN,NaN,NaN,...,"{""482015313003"":5,""482014211021"":4,""4816772230...",2021-03-29,3,2021,7,29,0,0,2,9
88454,zzw-225@8fc-f4b-hkf,sg:fc5b4618b40f4ec3bb452c7d31ee3265,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,NaN,NaN,NaN,...,"{""482015313003"":5,""482014211021"":4,""4816772230...",2021-03-30,3,2021,4,30,0,0,7,4


In [20]:
import os

for filename in os.listdir('/Users/mazcu/Downloads/safegraph_open_census_data/data/'): 
    print(filename)
    cosa = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/'+filename, encoding="utf-8")
    try:
        print(filename, cosa['B19013e1'])
    except:
        pass

In [89]:
cosa = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/cbg_b19.csv', encoding="utf-8")


In [63]:
df = subway

df['poi_cbg'] = df['poi_cbg'].map(int)
df['poi_cbg'] = df['poi_cbg'].map(str)


In [75]:
new = df[df['region'] == 'TX']
cbgs = set(new['poi_cbg'])

In [86]:
li = list(cosa['census_block_group'])
li = [str(i) for i in li]
for c in cbgs:

    if c in li:
        print('SÍ', c) #está    :D
    else:
        print('NO', c) #no está :,(

SÍ 482014323004
SÍ 482015517031
SÍ 482013501002
SÍ 482013330002
SÍ 482012220001
SÍ 482015401002
SÍ 482012329002
SÍ 482015340031
SÍ 482015410011
SÍ 482012323021
SÍ 482013402012
SÍ 482012214001
SÍ 482013119001
SÍ 482013112001
SÍ 482013332013
SÍ 482015217004
SÍ 482015510001
SÍ 482012227002
SÍ 482015530021
SÍ 482013128002
SÍ 482014107022
SÍ 482014301003
SÍ 482015508002
SÍ 482014401001
SÍ 482012520001
SÍ 482015417003
SÍ 482014543014
SÍ 482012313002
SÍ 482013109004
SÍ 482013340011
SÍ 482015114002
SÍ 482015336001
SÍ 482014532001
SÍ 482013338001
SÍ 482013319001
SÍ 482013126002
SÍ 482015328001
SÍ 482014211022
SÍ 482014319002
SÍ 482015515002
SÍ 482013321002
SÍ 482015522003
SÍ 482015432001
SÍ 482013125002
SÍ 482015409022
SÍ 482015408002
SÍ 482015527001
SÍ 482013102001
SÍ 482012115005
SÍ 482015204001
SÍ 482015115005
SÍ 482014229002
SÍ 482015321001
SÍ 482015341003
SÍ 482012117001
SÍ 482013339022
SÍ 482015514003
SÍ 482011000003
SÍ 482014521002
SÍ 482014217001
SÍ 482014505001
SÍ 482012226002
SÍ 48201